
## Extracting Named Entities (training) <br>

We use named entities (NEs) extracted from both translated title and translated body text. 

The extraction uses [spaCy API](https://spacy.io/).


### Extracting NEs from translated titles <br>

As one of our features, we extract named entities from the translated title using [spaCy.io](https://spacy.io/) and count the common ones. However, there are a few problems with title NE recognition:

>**Issue 1**. 
spaCy relies heavily on capitalization, so the first word will not be recognised:
eg. the first word in _"Taoiseach makes first general election campaign pitch in New Year video"_ will not be extracted

>**Solution**:
Adding "In fact," at the beginning of every title. However, if the first word of the title is not a NE, the capitalization will lead to spaCy recognising it as NE, but we chose to have an excess of NE than not have important NE.
---------------------------
>**Issue 2.**
Some sentences have all words capitalized (except prepositions and conjunctions), eg. "Zomato Buys Uber's Food Delivery Business in India in an All-Stock Deal", which leads to spaCy extracting _"Zomato Buys Uber's"_, _'Food Delivery Business'_, _'India'_ as spaCy relies heavily on capitalization

>**Solution:**
Truecasing with **standfordNLP's stanza** tool to truecase the sentence. However, from _"In fact, Zomato buys Uber's Food Delivery Business in India in an all - stock deal"_, spaCy still can only extract "India", not to mention running truecasing for every sentence (even the one already truecased) can lead to all nouns being capitalized, which affects spaCy greatly, and since when there's no NE (eg. "Warm For The Season For Now, Some Wet Snow This Weekend") no unwanted NE is extracted (we chose NOT to extract certain entities like 'CARDINAL','DATE', 'MONEY', 'ORDINAL', 'PERCENT', 'QUANTITY', 'TIME') , so we ended up decide not to truecase as we could gain very little while creating a new issues.
-------------------------
>**Issue 3.**
The extracted NE needs to be standardized in order to be compared

>**Solution:**
We lowercase them and removed "'s", "."("US" and "U.S.") and "-".
    

In [3]:
import pandas as pd
import spacy 
import stanza
# pip install stanza
stanza.download('en')
nlp = spacy.load('en_core_web_sm') 


2022-03-17 11:38:53 INFO: Downloading default packages for language: en (English)...


2022-03-17 11:38:55 INFO: File exists: /Users/chung-fantsai/stanza_resources/en/default.zip.
2022-03-17 11:39:00 INFO: Finished downloading models and saved to /Users/chung-fantsai/stanza_resources.


The chosen classes of named entities exclude the ones like _"CARDINAL"_, _"DATE"_, _"MONEY"_, _"ORDINAL"_, _"PERCENT"_, _"QUANTITY"_, and _"TIME"_. <br><br>

The reason for this is that the normalization of NEs in these categories might prove problematic. For instance, understanding that _"20/3/22"_ and _"the previous Sunday"_ correspond to the same date is not obvious. Moreover, we agreed that named entities, such as the ones referring to proper nouns of people or places, might be enough to capture the basic data of the story.

We give a short sample of truecasing to show our reasons for not employing it in our NER task.

In [ ]:
sentence = "Zomato Buys Uber's Food Delivery Business in India in an All-Stock Deal"
# ["Zomato Buys Uber's", 'Food Delivery Business', 'India']
# sentence = "in fact, zomato buys uber's food delivery business in india in an all-stock deal"
# # ["zomato buys uber's", 'india']
# sentence = "zomato buys uber's food delivery business in india in an all-stock deal"
# #["zomato buys uber's", 'india']
# sentence = "Zomato buys Uber's food delivery business in india in an all-stock deal"
# #['Uber', 'india']
# sentence = "Zomato buys Uber's food delivery business in india in an all-stock deal"
# # #['Uber', 'india']

doc = nlp(sentence) 
ignore_labels = ['CARDINAL','DATE', 'MONEY', 'ORDINAL', 'PERCENT', 'QUANTITY', 'TIME']
print([ent.text for ent in doc.ents  if ent.label_ not in ignore_labels])
for ent in doc.ents: 
    print(ent.text, ent.start_char, ent.end_char, ent.label_) 
    
def truecasing(text):
    stf_nlp = stanza.Pipeline(processors='tokenize,mwt,pos')
    doc = stf_nlp(text)
    return ' '.join([str(w.text.capitalize()) if w.upos in ["PROPN","NNS"] else str(w.text.lower()) for sent in doc.sentences for w in sent.words])

sentence = truecasing(sentence)
print("truecased: ", sentence)
doc = nlp(sentence) 

print([ent.text for ent in doc.ents  if ent.label_ not in ignore_labels])
for ent in doc.ents: 
    print(ent.text, ent.start_char, ent.end_char, ent.label_) 

In [5]:
title_df = pd.read_csv ('train/_TRAIN_title_translated.csv')
title_df

,pair_id,url1_lang,google_lang1,url2_lang,google_lang2,title1,title2,translated1,translated2
0,1484084337_1484110209,en,en,en,en,Virginia man arrested in fatal DUI crash in We...,Haiti’s leader marks independence day amid sec...,Virginia man arrested in fatal DUI crash in We...,Haiti’s leader marks independence day amid sec...
1,1484396422_1483924666,en,en,en,en,Guyana: Three injured after car crashes into u...,Fire kills more than 30 animals at zoo in west...,Guyana: Three injured after car crashes into u...,Fire kills more than 30 animals at zoo in west...
2,1484698254_1483758694,en,en,en,en,Trump Brings In 2020 At Mar-a-Lago: ‘We’re Goi...,"Trump says he does not expect war with Iran, ‘...",Trump Brings In 2020 At Mar-a-Lago: ‘We’re Goi...,"Trump says he does not expect war with Iran, ‘..."
3,1576314516_1576455088,en,en,en,en,Zomato Buys Uber’s Food Delivery Business in I...,Indian Online Food Delivery Market to Hit $8 B...,Zomato Buys Uber’s Food Delivery Business in I...,Indian Online Food Delivery Market to Hit $8 B...
4,1484036253_1483894099,en,en,en,en,"India approves third moon mission, months afte...",India targets new moon mission in 2020,"India approves third moon mission, months afte...",India targets new moon mission in 2020
...,...,...,...,...,...,...,...,...,...
4959,1586195445_1598778991,tr,tr,tr,tr,"BM, Aden'de 2 bini aşkın iç göçmenin selden za...",BM'den Yemen'de kadınların doğumda ölüm riski ...,"UN explained that over 2,000 domestic migrants...",Description of women's death risk warning at b...
4960,1590915424_1590940388,tr,tr,tr,tr,Kovid-19'dan dolayı La Liga kulüplerinde hayat...,Fabio Capello: Koronavirüs sonrası La Liga'da ...,Survival Struggle in La Liga Clubs due to Kovi...,Fabio Capello: After Coronavirus La Liga Only ...
4961,1526157103_1492737005,tr,tr,tr,tr,Saray da çare olmadı: 'Borca boğulan dev kulüp...,TFF’den jet yanıt! ''Bizi hedef gösteriyorlar'',The palace has not been the remedy: 'Giant Clu...,Jet response from TFF!'' They Target Us ''
4962,1603274500_1618292937,tr,tr,tr,tr,Ergene Belediyesi yol çalışmalarına aksatmadan...,Ergene'de Ahimehmet ve Yeşiltepe mahallelerind...,Ergene Municipality continues to disrupt road ...,Mask was distributed in Ahimehmet and Yeşiltep...



### Overlapping coefficient <br>

In the following we compute the overlap coefficient of the NEs in title pairs.  

Recall that for two sets A and B, the overlap coefficient is defined as:

$$
OS(A,B) = \frac{|A \cap B|}{max(|A|,|B))}
$$

for $|A|$ denoting the cardinality of the set $A$. <br><br>

The reason for choosing the overlap coefficient as similarity score (rather than Dice coefficient or [Jaccard similarity](https://en.wikipedia.org/wiki/Jaccard_index)) is that it is not strongly influenced by a large difference in size of the two sets.  

For instance, if a title has 2 NEs and a second title has 20, the maximum value for the Jaccard similarity would be 10% similarity, whereas the overlapping coefficient would still show 100% similarity as maximum possible value. 

In [ ]:

def overlapScore(key1, key2):
    key1 = set(key1)
    key2 = set(key2)
    interset = key1.intersection(key2)
    union = key1.union(key2)
    
    denominator = len(key1)
    if len(key2) < denominator:
        denominator = len(key2)
    
    if len(key1) <1 or len(key2) <1 or len(union) < 1 or len(interset)<1:
        return 0
    return (len(interset)/(denominator))

title_key_df = pd.DataFrame(columns = ["pair_id","url1_lang", "google_lang1","url2_lang","google_lang2", "title1","title2","key1","key2", "key_score"])

for index, row in title_df.iterrows(): 
    
    #solve Issue 1 by adding 'In fact, '
    issue1_solve = 'In fact, '
    title1 = issue1_solve + row['translated1']
    title2 = issue1_solve + row['translated2']
    doc1 = nlp(title1) 
    doc2 = nlp(title2) 
    ner1 = [(((ent.text.lower()).replace("'s", "")).replace("-"," ")).replace(".","") for ent in doc1.ents if ent.label_ not in ignore_labels]
    ner2 = [(((ent.text.lower()).replace("'s", "")).replace("-"," ")).replace(".","") for ent in doc2.ents if ent.label_ not in ignore_labels]
    
    score = overlapScore(ner1, ner2)
    pair = row['pair_id']
    if score >0:
        print('---------------------',index,'-----------------------')
        print(pair)
        print(title1)
        print(ner1)
        print(title2)
        print(ner2)
        print(score)
    
    
    entry = {"pair_id": pair,"url1_lang":row['url1_lang'],"google_lang1":row['google_lang1'],"url2_lang":row['url2_lang'],"google_lang2":row['google_lang2'],"title1":row["title1"],"title2":row["title2"],"translated1":row['translated1'],"translated2":row['translated2'],"key1":ner1,"key2":ner2, "key_score":score}
    title_key_df = title_key_df.append(entry, ignore_index = True)
    

C:\Users\Potsdam\AppData\Local\Temp\ipykernel_29252\1006212408.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  title_key_df = title_key_df.append(entry, ignore_index = True)
C:\Users\Potsdam\AppData\Local\Temp\ipykernel_29252\1006212408.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  title_key_df = title_key_df.append(entry, ignore_index = True)
C:\Users\Potsdam\AppData\Local\Temp\ipykernel_29252\1006212408.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  title_key_df = title_key_df.append(entry, ignore_index = True)
C:\Users\Potsdam\AppData\Local\Temp\ipykernel_29252\1006212408.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

--------------------- 4 -----------------------
1484036253_1483894099
In fact, India approves third moon mission, months after landing failure
['india']
In fact, India targets new moon mission in 2020
['india']
1.0
--------------------- 8 -----------------------
1484011751_1483920335
In fact, The Compatriots: The Brutal and Chaotic History of Russia's Exiles, Émigrés, and Agents Abroad: Soldatov, Andrei, Borogan, Irina: 9781541730168: Amazon.com: Books
['russia', 'exiles, émigrés', 'andrei', 'borogan', 'irina', 'amazoncom: books']
In fact, The Red Web: The Kremlin's Wars on the Internet: Soldatov, Andrei, Borogan, Irina: 9781610399579: Amazon.com: Books
['kremlin', 'soldatov', 'andrei', 'borogan', 'irina', 'amazoncom: books']
0.5


C:\Users\Potsdam\AppData\Local\Temp\ipykernel_29252\1006212408.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  title_key_df = title_key_df.append(entry, ignore_index = True)
C:\Users\Potsdam\AppData\Local\Temp\ipykernel_29252\1006212408.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  title_key_df = title_key_df.append(entry, ignore_index = True)
C:\Users\Potsdam\AppData\Local\Temp\ipykernel_29252\1006212408.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  title_key_df = title_key_df.append(entry, ignore_index = True)
C:\Users\Potsdam\AppData\Local\Temp\ipykernel_29252\1006212408.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

--------------------- 17 -----------------------
1484033615_1483970348
In fact, TMC observes foundation day as ‘Citizens’ Day’, vows to continue protests till CAA is withdrawn
['caa']
In fact, Actor Swara Bhasker lauds Jamia students for waking up entire country against CAA
['swara bhasker', 'caa']
0.5


C:\Users\Potsdam\AppData\Local\Temp\ipykernel_29252\1006212408.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  title_key_df = title_key_df.append(entry, ignore_index = True)
C:\Users\Potsdam\AppData\Local\Temp\ipykernel_29252\1006212408.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  title_key_df = title_key_df.append(entry, ignore_index = True)
C:\Users\Potsdam\AppData\Local\Temp\ipykernel_29252\1006212408.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  title_key_df = title_key_df.append(entry, ignore_index = True)
C:\Users\Potsdam\AppData\Local\Temp\ipykernel_29252\1006212408.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

...

C:\Users\Potsdam\AppData\Local\Temp\ipykernel_29252\1006212408.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  title_key_df = title_key_df.append(entry, ignore_index = True)
C:\Users\Potsdam\AppData\Local\Temp\ipykernel_29252\1006212408.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  title_key_df = title_key_df.append(entry, ignore_index = True)
C:\Users\Potsdam\AppData\Local\Temp\ipykernel_29252\1006212408.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  title_key_df = title_key_df.append(entry, ignore_index = True)
C:\Users\Potsdam\AppData\Local\Temp\ipykernel_29252\1006212408.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in


In fact, Press voice: 'Badische Newest News' to 'Coronavirus'
["badische newest news'", 'coronavirus']
0.5
--------------------- 3940 -----------------------
1512852781_1495990721
In fact, Swedish defender for Kloten - Hoefner Volksblatt and Marchanzeiger
['swedish', 'kloten   hoefner volksblatt', 'marchanzeiger']
In fact, Gck Lions stay against Kloten Sieglos - Hoefner Volksblatt and Marchanzeiger
['kloten sieglos   hoefner', 'volksblatt', 'marchanzeiger']
0.2
--------------------- 3944 -----------------------
1636071284_1632845395
In fact, Africa wants to fight virus with "community approach": that is behind it
['africa']
In fact, Coronavirus - «Up to 44 million infected»: First spared from the pandemic, Corona spreads faster in Africa
['corona', 'africa']
0.5


C:\Users\Potsdam\AppData\Local\Temp\ipykernel_29252\1006212408.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  title_key_df = title_key_df.append(entry, ignore_index = True)
C:\Users\Potsdam\AppData\Local\Temp\ipykernel_29252\1006212408.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  title_key_df = title_key_df.append(entry, ignore_index = True)
C:\Users\Potsdam\AppData\Local\Temp\ipykernel_29252\1006212408.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  title_key_df = title_key_df.append(entry, ignore_index = True)
C:\Users\Potsdam\AppData\Local\Temp\ipykernel_29252\1006212408.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

--------------------- 3949 -----------------------
1631007501_1630829939
In fact, Border controls in the EU ends 16 June
['eu']
In fact, From 16 June: EU citizens are allowed to enter: border controls are finished again
['eu']
1.0
--------------------- 3951 -----------------------
1623131784_1629138912
In fact, Italy launches its Corona warning app
['italy', 'corona']
In fact, Italy: 65 Corona death victims in 24 hours
['italy', 'corona']
1.0
--------------------- 3953 -----------------------
1591941289_1576889083
In fact, Bund-Country Round: No larger Corona relaxations to be expected
['bund country round', 'corona']
In fact, Corona relaxations: CDU Chef Kramp-Karrenbauer calls for a uniform approach
['corona', 'cdu', 'chef kramp karrenbauer']
0.25


C:\Users\Potsdam\AppData\Local\Temp\ipykernel_29252\1006212408.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  title_key_df = title_key_df.append(entry, ignore_index = True)
C:\Users\Potsdam\AppData\Local\Temp\ipykernel_29252\1006212408.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  title_key_df = title_key_df.append(entry, ignore_index = True)
C:\Users\Potsdam\AppData\Local\Temp\ipykernel_29252\1006212408.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  title_key_df = title_key_df.append(entry, ignore_index = True)
C:\Users\Potsdam\AppData\Local\Temp\ipykernel_29252\1006212408.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

In [71]:
title_key_df

,pair_id,url1_lang,google_lang1,url2_lang,google_lang2,title1,title2,key1,key2,key_score,translated1,translated2
0,1484084337_1484110209,en,en,en,en,Virginia man arrested in fatal DUI crash in We...,Haiti’s leader marks independence day amid sec...,"['virginia', 'west virginia']",['haiti'],0.0,Virginia man arrested in fatal DUI crash in We...,Haiti’s leader marks independence day amid sec...
1,1484396422_1483924666,en,en,en,en,Guyana: Three injured after car crashes into u...,Fire kills more than 30 animals at zoo in west...,['guyana'],['germany'],0.0,Guyana: Three injured after car crashes into u...,Fire kills more than 30 animals at zoo in west...
2,1484698254_1483758694,en,en,en,en,Trump Brings In 2020 At Mar-a-Lago: ‘We’re Goi...,"Trump says he does not expect war with Iran, ‘...",['trump brings'],"['trump', 'iran']",0.0,Trump Brings In 2020 At Mar-a-Lago: ‘We’re Goi...,"Trump says he does not expect war with Iran, ‘..."
3,1576314516_1576455088,en,en,en,en,Zomato Buys Uber’s Food Delivery Business in I...,Indian Online Food Delivery Market to Hit $8 B...,"['zomato buys uber’s', 'india']","['indian', 'bcg report']",0.0,Zomato Buys Uber’s Food Delivery Business in I...,Indian Online Food Delivery Market to Hit $8 B...
4,1484036253_1483894099,en,en,en,en,"India approves third moon mission, months afte...",India targets new moon mission in 2020,['india'],['india'],1.0,"India approves third moon mission, months afte...",India targets new moon mission in 2020
...,...,...,...,...,...,...,...,...,...,...,...,...
4959,1586195445_1598778991,tr,tr,tr,tr,"BM, Aden'de 2 bini aşkın iç göçmenin selden za...",BM'den Yemen'de kadınların doğumda ölüm riski ...,"['un', 'aden']","['yemen', 'un']",0.5,"UN explained that over 2,000 domestic migrants...",Description of women's death risk warning at b...
4960,1590915424_1590940388,tr,tr,tr,tr,Kovid-19'dan dolayı La Liga kulüplerinde hayat...,Fabio Capello: Koronavirüs sonrası La Liga'da ...,"['survival struggle', 'la liga clubs']","['fabio capello', 'coronavirus la liga only re...",0.0,Survival Struggle in La Liga Clubs due to Kovi...,Fabio Capello: After Coronavirus La Liga Only ...
4961,1526157103_1492737005,tr,tr,tr,tr,Saray da çare olmadı: 'Borca boğulan dev kulüp...,TFF’den jet yanıt! ''Bizi hedef gösteriyorlar'',['giant clubs'],"['jet', 'tff']",0.0,The palace has not been the remedy: 'Giant Clu...,Jet response from TFF!'' They Target Us ''
4962,1603274500_1618292937,tr,tr,tr,tr,Ergene Belediyesi yol çalışmalarına aksatmadan...,Ergene'de Ahimehmet ve Yeşiltepe mahallelerind...,['ergene municipality'],"['mask', 'ergene']",0.0,Ergene Municipality continues to disrupt road ...,Mask was distributed in Ahimehmet and Yeşiltep...


In [75]:
path = 'train/_TRAIN_title_named-entity_score.csv'
title_key_df.to_csv(path,index=False)

In [72]:
title_key_df.at[8,'key_score']

0.6666666666666666

In [22]:
len(title_key_df[title_key_df['key_score']!=0])

865

In [25]:
len(title_key_df[title_key_df['key_score']>0.5])

607

In [26]:
len(title_key_df[title_key_df['key_score']==1])

583

The overlap coefficient is close to 1 whenever the NEs are similar, close to 0 if they are dissimilar. <br>

As we observed, there are 865 pairs with non-zero score, 607 whose score is between 0.5 and 1, and, finally, 583 whose score is precisely 1. <br>

In what follows, we compare the similarity score obtained by extracting NEs from the titles and we compare it to the annotated *'Overall'* similarity.

In [52]:
import math

# see how indicating title named entities are to the similairty of two articles
train_df = pd.read_csv ('train/_TRAIN_details_in_df.csv')

def normal_round(n):
    if n - math.floor(n) < 0.5:
        return int(math.floor(n))
    return int(math.ceil(n))

compare_df = pd.DataFrame(columns = ["overall","title_ne_score"])


for i, row in train_df.iterrows(): 
#     if row["pair_id"] != title_key_df.iloc[i]['pair_id']:
#         print('---------------------',i,'-----------------------')
#         print(row["pair_id"])
    print('---------------------',i,'-----------------------')
    label = normal_round(row['Overall'])
    print('label:',label)
    score = title_key_df.iloc[i]['key_score']
    print('score:',score)
    
    entry = {"overall":label,"title_ne_score":score}
    compare_df = compare_df.append(entry, ignore_index = True)
    

--------------------- 0 -----------------------
label: 4
score: 0.0
--------------------- 1 -----------------------
label: 4
score: 0.0
--------------------- 2 -----------------------
label: 2
score: 0.0
--------------------- 3 -----------------------
label: 2
score: 0.0
--------------------- 4 -----------------------
label: 1
score: 1.0
--------------------- 5 -----------------------
label: 2
score: 0.0
--------------------- 6 -----------------------
label: 4
score: 0.0
--------------------- 7 -----------------------
label: 3
score: 0.0
--------------------- 8 -----------------------
label: 4
score: 0.6666666666666666
--------------------- 9 -----------------------
label: 3
score: 0.0
--------------------- 10 -----------------------
label: 3
score: 0.0
--------------------- 11 -----------------------
label: 4
score: 0.0
--------------------- 12 -----------------------
label: 4
score: 0.0
--------------------- 13 -----------------------
label: 4
score: 0.0
--------------------- 14 ----

--------------------- 162 -----------------------
label: 4
score: 1.0
--------------------- 163 -----------------------
label: 4
score: 0.0
--------------------- 164 -----------------------
label: 4
score: 0.0
--------------------- 165 -----------------------
label: 4
score: 0.0
--------------------- 166 -----------------------
label: 1
score: 1.0
--------------------- 167 -----------------------
label: 4
score: 0.0
--------------------- 168 -----------------------
label: 1
score: 0.0
--------------------- 169 -----------------------
label: 2
score: 1.0
--------------------- 170 -----------------------
label: 4
score: 0.0
--------------------- 171 -----------------------
label: 4
score: 0.0
--------------------- 172 -----------------------
label: 4
score: 0.0
--------------------- 173 -----------------------
label: 4
score: 0.0
--------------------- 174 -----------------------
label: 3
score: 0.0
--------------------- 175 -----------------------
label: 4
score: 0.0
--------------------

--------------------- 305 -----------------------
label: 4
score: 0.0
--------------------- 306 -----------------------
label: 3
score: 0.0
--------------------- 307 -----------------------
label: 4
score: 0.0
--------------------- 308 -----------------------
label: 1
score: 0.5
--------------------- 309 -----------------------
label: 4
score: 0.0
--------------------- 310 -----------------------
label: 4
score: 0.0
--------------------- 311 -----------------------
label: 4
score: 0.0
--------------------- 312 -----------------------
label: 4
score: 0.0
--------------------- 313 -----------------------
label: 4
score: 0.0
--------------------- 314 -----------------------
label: 3
score: 1.0
--------------------- 315 -----------------------
label: 4
score: 0.0
--------------------- 316 -----------------------
label: 3
score: 0.0
--------------------- 317 -----------------------
label: 2
score: 0.0
--------------------- 318 -----------------------
label: 4
score: 0.0
--------------------

--------------------- 464 -----------------------
label: 4
score: 0.0
--------------------- 465 -----------------------
label: 4
score: 0.0
--------------------- 466 -----------------------
label: 1
score: 0.0
--------------------- 467 -----------------------
label: 1
score: 0.3333333333333333
--------------------- 468 -----------------------
label: 3
score: 0.0
--------------------- 469 -----------------------
label: 4
score: 0.0
--------------------- 470 -----------------------
label: 1
score: 0.0
--------------------- 471 -----------------------
label: 1
score: 0.3333333333333333
--------------------- 472 -----------------------
label: 4
score: 0.0
--------------------- 473 -----------------------
label: 2
score: 0.0
--------------------- 474 -----------------------
label: 1
score: 0.0
--------------------- 475 -----------------------
label: 4
score: 0.0
--------------------- 476 -----------------------
label: 4
score: 0.0
--------------------- 477 -----------------------
label: 4
s

--------------------- 634 -----------------------
label: 4
score: 0.0
--------------------- 635 -----------------------
label: 2
score: 0.0
--------------------- 636 -----------------------
label: 3
score: 0.0
--------------------- 637 -----------------------
label: 4
score: 0.0
--------------------- 638 -----------------------
label: 4
score: 0.0
--------------------- 639 -----------------------
label: 2
score: 1.0
--------------------- 640 -----------------------
label: 4
score: 0.3333333333333333
--------------------- 641 -----------------------
label: 3
score: 0.0
--------------------- 642 -----------------------
label: 3
score: 0.0
--------------------- 643 -----------------------
label: 4
score: 0.0
--------------------- 644 -----------------------
label: 2
score: 1.0
--------------------- 645 -----------------------
label: 1
score: 1.0
--------------------- 646 -----------------------
label: 2
score: 0.0
--------------------- 647 -----------------------
label: 4
score: 0.0
-----

...

label: 1
score: 0.0
--------------------- 4189 -----------------------
label: 3
score: 0.5
--------------------- 4190 -----------------------
label: 3
score: 0.0
--------------------- 4191 -----------------------
label: 2
score: 0.0
--------------------- 4192 -----------------------
label: 3
score: 0.0
--------------------- 4193 -----------------------
label: 2
score: 1.0
--------------------- 4194 -----------------------
label: 2
score: 0.0
--------------------- 4195 -----------------------
label: 3
score: 0.0
--------------------- 4196 -----------------------
label: 3
score: 0.0
--------------------- 4197 -----------------------
label: 2
score: 0.0
--------------------- 4198 -----------------------
label: 3
score: 0.0
--------------------- 4199 -----------------------
label: 3
score: 0.0
--------------------- 4200 -----------------------
label: 2
score: 0.0
--------------------- 4201 -----------------------
label: 2
score: 0.0
--------------------- 4202 -----------------------
label:

--------------------- 4340 -----------------------
label: 2
score: 0.0
--------------------- 4341 -----------------------
label: 4
score: 0.0
--------------------- 4342 -----------------------
label: 4
score: 0.0
--------------------- 4343 -----------------------
label: 4
score: 0.0
--------------------- 4344 -----------------------
label: 4
score: 0.0
--------------------- 4345 -----------------------
label: 3
score: 1.0
--------------------- 4346 -----------------------
label: 4
score: 0.0
--------------------- 4347 -----------------------
label: 4
score: 0.0
--------------------- 4348 -----------------------
label: 4
score: 0.0
--------------------- 4349 -----------------------
label: 4
score: 0.0
--------------------- 4350 -----------------------
label: 4
score: 0.0
--------------------- 4351 -----------------------
label: 4
score: 0.0
--------------------- 4352 -----------------------
label: 4
score: 0.0
--------------------- 4353 -----------------------
label: 1
score: 0.0
------

--------------------- 4494 -----------------------
label: 4
score: 0.0
--------------------- 4495 -----------------------
label: 1
score: 0.0
--------------------- 4496 -----------------------
label: 4
score: 0.0
--------------------- 4497 -----------------------
label: 2
score: 1.0
--------------------- 4498 -----------------------
label: 4
score: 0.0
--------------------- 4499 -----------------------
label: 4
score: 0.0
--------------------- 4500 -----------------------
label: 4
score: 0.0
--------------------- 4501 -----------------------
label: 4
score: 1.0
--------------------- 4502 -----------------------
label: 1
score: 0.0
--------------------- 4503 -----------------------
label: 3
score: 0.0
--------------------- 4504 -----------------------
label: 1
score: 0.0
--------------------- 4505 -----------------------
label: 3
score: 0.0
--------------------- 4506 -----------------------
label: 4
score: 0.0
--------------------- 4507 -----------------------
label: 4
score: 0.0
------

score: 1.0
--------------------- 4637 -----------------------
label: 4
score: 1.0
--------------------- 4638 -----------------------
label: 2
score: 0.25
--------------------- 4639 -----------------------
label: 4
score: 0.5
--------------------- 4640 -----------------------
label: 4
score: 0.0
--------------------- 4641 -----------------------
label: 4
score: 1.0
--------------------- 4642 -----------------------
label: 3
score: 0.0
--------------------- 4643 -----------------------
label: 3
score: 0.0
--------------------- 4644 -----------------------
label: 2
score: 1.0
--------------------- 4645 -----------------------
label: 2
score: 0.0
--------------------- 4646 -----------------------
label: 3
score: 0.0
--------------------- 4647 -----------------------
label: 4
score: 0.0
--------------------- 4648 -----------------------
label: 4
score: 0.0
--------------------- 4649 -----------------------
label: 4
score: 0.0
--------------------- 4650 -----------------------
label: 1
score

--------------------- 4784 -----------------------
label: 1
score: 1.0
--------------------- 4785 -----------------------
label: 3
score: 0.0
--------------------- 4786 -----------------------
label: 3
score: 0.0
--------------------- 4787 -----------------------
label: 2
score: 0.0
--------------------- 4788 -----------------------
label: 3
score: 1.0
--------------------- 4789 -----------------------
label: 3
score: 0.0
--------------------- 4790 -----------------------
label: 4
score: 0.0
--------------------- 4791 -----------------------
label: 4
score: 0.0
--------------------- 4792 -----------------------
label: 3
score: 0.0
--------------------- 4793 -----------------------
label: 3
score: 0.0
--------------------- 4794 -----------------------
label: 4
score: 0.0
--------------------- 4795 -----------------------
label: 4
score: 0.0
--------------------- 4796 -----------------------
label: 4
score: 0.5
--------------------- 4797 -----------------------
label: 2
score: 0.5
------

In [54]:
compare_4 = compare_df[compare_df['overall']==4.0]
compare_3 = compare_df[compare_df['overall']==3.0]
compare_2 = compare_df[compare_df['overall']==2.0]
compare_1 = compare_df[compare_df['overall']==1.0]
len(compare_df)
len(compare_4) + len(compare_3) + len(compare_2) +len(compare_1)

i_list = [1.0,2.0,3.0,4.0]
compare = [compare_1,compare_2,compare_3,compare_4]
for i in i_list:
    index = int(i)-1
    c = compare[index]
    print('Number of pairs that are rated as ',i,':')
    print(len(c))
    print('Pairs that are rated as ',i,', title named entity score is 0: (value, percentage)')
    print(len(c[c['title_ne_score']==0]), ', ', len(c[c['title_ne_score']==0])/len(c)*100,'%')
    print('Pairs that are rated as ',i,', title named entity score is > 0.3:')
    print(len(c[c['title_ne_score']>0.3]), ', ', len(c[c['title_ne_score']>0.3])/len(c)*100,'%')
    print('Pairs that are rated as ',i,', title named entity score is > 0.5:')
    print(len(c[c['title_ne_score']>0.5]), ', ', len(c[c['title_ne_score']>0.5])/len(c)*100,'%')
    print('Pairs that are rated as ',i,', title named entity score is 1.0:')
    print(len(c[c['title_ne_score']==1.0]), ', ', len(c[c['title_ne_score']==1.0])/len(c)*100,'%')

Number of pairs that are rated as  1.0 :
963
Pairs that are rated as  1.0 , title named entity score is 0: (value, percentage)
628 ,  65.2128764278297 %
Pairs that are rated as  1.0 , title named entity score is > 0.3:
333 ,  34.57943925233645 %
Pairs that are rated as  1.0 , title named entity score is > 0.5:
240 ,  24.922118380062305 %
Pairs that are rated as  1.0 , title named entity score is 1.0:
228 ,  23.67601246105919 %
Number of pairs that are rated as  2.0 :
957
Pairs that are rated as  2.0 , title named entity score is 0: (value, percentage)
715 ,  74.71264367816092 %
Pairs that are rated as  2.0 , title named entity score is > 0.3:
239 ,  24.973876698014628 %
Pairs that are rated as  2.0 , title named entity score is > 0.5:
160 ,  16.718913270637408 %
Pairs that are rated as  2.0 , title named entity score is 1.0:
151 ,  15.778474399164056 %
Number of pairs that are rated as  3.0 :
1067
Pairs that are rated as  3.0 , title named entity score is 0: (value, percentage)
911 ,  


### Remarks: <br>

As we expected, it's hard to tell if extracting NE from titles is a good feature to decide whether two articles are telling the same event. The reason being that some titles are NOT very descriptive, instead some tend to be sensationalistic or provocative, completely disregarding facts that we can extract NEs from. Even when two titles are associated to the same event, different news platforms may adopt different titling strategies; either they choose to keep it very short, or very long, or use an idiom, a phrase, a joke, or puns, making titling very much a stylistic choice. This is reasonable if one thinks that titles should be as striking and as different from one another as possible in order to be succesful. <br><br>

One argument against the use of title NEs as a feature is that  named entities appearing in the title are also likely be mentioned in the text. Since we are also extracting NEs from the text and comparing them in pairs, it can be argued that having title named entity similarity score is adding a similar feature that might not be necessary.

Nevertheless, from the previous short analysis, we can see a tendency when a pair is similar (rated as 1.0 or not much smaller), the high named entity similarity score occurs more common (23% of the title named entity have a score of precisely 1.0 for the most similar pairs), and 5% of them are scored 1.0 for the least similar pairs (rated 4.0))<br><br>

To get a better judgement whether or not this feature is necessary, we also examine the similarity score of the named entity extracted from the body text of the article.


### Extracting Named Entities from the body text <br>

The script used for translation and NE extraction of the body text is in `\07_calculate_training_text_named_entity_similarity_score.ipynb`

In [56]:
# so we input the text named entity dataframe we did in another notebook that contains the named entity similarity score
text_ne_df = pd.read_csv ('train/_TRAIN_text_named-entity_score.csv')
text_ne_df

,pair_id,url1_lang,url2_lang,text1,text2,translated_body1,translated_body2,key1,key2,key_score
0,1484084337_1484110209,en,en,"MARTINSBURG, W.Va. — A suspected drunken drive...","PORT-AU-PRINCE, Haiti — Haitian President Jove...","MARTINSBURG, W.Va. — A suspected drunken drive...","PORT-AU-PRINCE, Haiti — Haitian President Jove...","['martinsburg', 'wva', 'new year’s day', 'west...","['haiti', 'haitian', 'jovenel moïse', 'moïse',...",0.111111
1,1484396422_1483924666,en,en,Share This On:\n\nPin 11 Shares\n\n(NEWS ROOM ...,BERLIN - A fire at a zoo in western Germany in...,Share This On: Pin 11 Shares (NEWS ROOM GUYA...,BERLIN - A fire at a zoo in western Germany in...,"['the crabwood creek public road', 'new year’s...","['germany', 'krefeld', 'gerd hoppmann', 'hoppm...",0.000000
2,1484698254_1483758694,en,en,(Breitbart) – President Donald Trump welcomed ...,"PALM BEACH, United States — US President Donal...",(Breitbart) – President Donald Trump welcomed ...,"PALM BEACH, United States — US President Donal...","['breitbart', 'donald trump', 'new year’s eve'...","['palm beach', 'united states', 'us', 'donald ...",0.300000
3,1576314516_1576455088,en,en,Uber has sold its online food-ordering busines...,Rapid digitisation and growth in both online b...,Uber has sold its online food-ordering busines...,Rapid digitisation and growth in both online b...,"['india', 'zomato', 'china', 'ant financial', ...","['india', 'google', 'boston consulting group',...",0.263158
4,1484036253_1483894099,en,en,BENGALURU (Reuters) - India has approved its t...,BANGALORE: India plans to make a fresh attempt...,BENGALURU (Reuters) - India has approved its t...,BANGALORE: India plans to make a fresh attempt...,"['bengaluru', 'reuters', 'india', 'indian spac...","['india', 'indian space research organisation'...",0.777778
...,...,...,...,...,...,...,...,...,...,...
4959,1586195445_1598778991,tr,tr,"BM, Aden'de 2 bini aşkın iç göçmenin selden za...",BM'den Yemen'de kadınların doğumda ölüm riski ...,"The UN announced that more than 2,000 domestic...","In Yemen, Women's Death Risk Warning Explanati...","['un', 'aden', 'the united nations', 'un', 'ye...","['yemen', 'women death risk warning explanatio...",0.600000
4960,1590915424_1590940388,tr,tr,Kovid-19'dan dolayı La Liga kulüplerinde hayat...,Yeni tip koronavirüs (Kovid-19) salgınının eko...,"Because of the Kovid-19, the Survival Struggle...",The new type of coronavirus (Kovid-19) is cons...,"['la liga clubs', 'football league', 'la liga'...","['football league', 'la liga', 'spanish', 'the...",1.000000
4961,1526157103_1492737005,tr,tr,\n\n\n\n\n\n\n\nİflas noktasındaki kulüplerin ...,"TFF, resmi internet sitesinden Beşiktaş'ın fai...",It is stated that the sales of the clubs at th...,TFF has published an explanation on the offici...,"['besiktas', 'ahmet nur dek', 'turkey banks as...","['tff', 'besiktas', 'team spend limits', 'ahme...",0.333333
4962,1603274500_1618292937,tr,tr,Ergene Belediyesi yol çalışmalarına aksatmadan...,Ergene'de Ahimehmet ve Yeşiltepe mahallelerind...,Ergene Municipality continues without disrupti...,"In Ergene, the mask was distributed in Ahimehm...","['ergene municipality', 'tekirdag', 'ergene mu...","['ergene', 'ergene county', 'ergene', 'ergene'...",0.300000


In [68]:
# get a dataframe showing the distribution of label (annotated overalll similarity) and our text named entity similarity score

compare_df = pd.DataFrame(columns = ["overall","text_ne_score"])


for i, row in train_df.iterrows(): 
#     if row["pair_id"] != title_key_df.iloc[i]['pair_id']:
#         print('---------------------',i,'-----------------------')
#         print(row["pair_id"])
    print('---------------------',i,'-----------------------')
    label = normal_round(row['Overall'])
    print('label:',label)
    score = text_ne_df.iloc[i]['key_score']
    print('score:',score)
    
    entry = {"overall":label,"text_ne_score":score}
    compare_df = compare_df.append(entry, ignore_index = True)
    

--------------------- 0 -----------------------
label: 4
score: 0.1111111111111111
--------------------- 1 -----------------------
label: 4
score: 0.0
--------------------- 2 -----------------------
label: 2
score: 0.3
--------------------- 3 -----------------------
label: 2
score: 0.2631578947368421
--------------------- 4 -----------------------
label: 1
score: 0.7777777777777778
--------------------- 5 -----------------------
label: 2
score: 0.4074074074074074
--------------------- 6 -----------------------
label: 4
score: 0.0
--------------------- 7 -----------------------
label: 3
score: 0.07692307692307693
--------------------- 8 -----------------------
label: 4
score: 0.4444444444444444
--------------------- 9 -----------------------
label: 3
score: 0.16666666666666666
--------------------- 10 -----------------------
label: 3
score: 0.0
--------------------- 11 -----------------------
label: 4
score: 0.2631578947368421
--------------------- 12 -----------------------
label: 4
sc

--------------------- 124 -----------------------
label: 1
score: 0.5217391304347826
--------------------- 125 -----------------------
label: 4
score: 0.0
--------------------- 126 -----------------------
label: 1
score: 0.3157894736842105
--------------------- 127 -----------------------
label: 4
score: 0.0
--------------------- 128 -----------------------
label: 3
score: 0.4
--------------------- 129 -----------------------
label: 4
score: 0.0
--------------------- 130 -----------------------
label: 4
score: 0.045454545454545456
--------------------- 131 -----------------------
label: 4
score: 0.1111111111111111
--------------------- 132 -----------------------
label: 3
score: 0.5555555555555556
--------------------- 133 -----------------------
label: 3
score: 0.0
--------------------- 134 -----------------------
label: 3
score: 0.0
--------------------- 135 -----------------------
label: 2
score: 0.35
--------------------- 136 -----------------------
label: 4
score: 0.0
------------

score: 0.0
--------------------- 266 -----------------------
label: 4
score: 0.1111111111111111
--------------------- 267 -----------------------
label: 1
score: 0.4444444444444444
--------------------- 268 -----------------------
label: 4
score: 0.3333333333333333
--------------------- 269 -----------------------
label: 4
score: 0.0
--------------------- 270 -----------------------
label: 2
score: 0.4545454545454545
--------------------- 271 -----------------------
label: 4
score: 0.0
--------------------- 272 -----------------------
label: 2
score: 0.0
--------------------- 273 -----------------------
label: 4
score: 0.0
--------------------- 274 -----------------------
label: 4
score: 0.0
--------------------- 275 -----------------------
label: 3
score: 0.17391304347826084
--------------------- 276 -----------------------
label: 4
score: 0.25
--------------------- 277 -----------------------
label: 3
score: 0.2
--------------------- 278 -----------------------
label: 4
score: 0.0
--

--------------------- 405 -----------------------
label: 3
score: 0.28
--------------------- 406 -----------------------
label: 4
score: 0.3333333333333333
--------------------- 407 -----------------------
label: 4
score: 0.0
--------------------- 408 -----------------------
label: 4
score: 0.0
--------------------- 409 -----------------------
label: 2
score: 0.5
--------------------- 410 -----------------------
label: 4
score: 0.26666666666666666
--------------------- 411 -----------------------
label: 4
score: 0.1111111111111111
--------------------- 412 -----------------------
label: 4
score: 0.25
--------------------- 413 -----------------------
label: 3
score: 0.0
--------------------- 414 -----------------------
label: 4
score: 0.0
--------------------- 415 -----------------------
label: 1
score: 0.4705882352941176
--------------------- 416 -----------------------
label: 1
score: 0.25
--------------------- 417 -----------------------
label: 3
score: 0.0
--------------------- 418 

--------------------- 576 -----------------------
label: 1
score: 0.5428571428571428
--------------------- 577 -----------------------
label: 3
score: 0.6666666666666666
--------------------- 578 -----------------------
label: 4
score: 0.0
--------------------- 579 -----------------------
label: 4
score: 0.027777777777777776
--------------------- 580 -----------------------
label: 4
score: 0.1111111111111111
--------------------- 581 -----------------------
label: 3
score: 0.0
--------------------- 582 -----------------------
label: 4
score: 0.125
--------------------- 583 -----------------------
label: 4
score: 0.1111111111111111
--------------------- 584 -----------------------
label: 4
score: 0.0
--------------------- 585 -----------------------
label: 3
score: 0.0
--------------------- 586 -----------------------
label: 4
score: 0.0
--------------------- 587 -----------------------
label: 1
score: 0.8
--------------------- 588 -----------------------
label: 4
score: 0.3846153846153

...

--------------------- 4412 -----------------------
label: 4
score: 0.25
--------------------- 4413 -----------------------
label: 3
score: 0.0
--------------------- 4414 -----------------------
label: 4
score: 0.0
--------------------- 4415 -----------------------
label: 4
score: 0.0
--------------------- 4416 -----------------------
label: 4
score: 0.25
--------------------- 4417 -----------------------
label: 3
score: 0.25
--------------------- 4418 -----------------------
label: 4
score: 0.0
--------------------- 4419 -----------------------
label: 3
score: 0.5
--------------------- 4420 -----------------------
label: 3
score: 0.6428571428571429
--------------------- 4421 -----------------------
label: 4
score: 0.0
--------------------- 4422 -----------------------
label: 1
score: 0.5142857142857142
--------------------- 4423 -----------------------
label: 4
score: 0.4545454545454545
--------------------- 4424 -----------------------
label: 3
score: 0.9
--------------------- 4425 --

--------------------- 4575 -----------------------
label: 4
score: 0.16666666666666666
--------------------- 4576 -----------------------
label: 4
score: 0.0
--------------------- 4577 -----------------------
label: 4
score: 0.0
--------------------- 4578 -----------------------
label: 4
score: 0.5
--------------------- 4579 -----------------------
label: 4
score: 0.6
--------------------- 4580 -----------------------
label: 2
score: 0.4
--------------------- 4581 -----------------------
label: 3
score: 0.5555555555555556
--------------------- 4582 -----------------------
label: 4
score: 0.0
--------------------- 4583 -----------------------
label: 4
score: 0.42857142857142855
--------------------- 4584 -----------------------
label: 4
score: 0.09090909090909093
--------------------- 4585 -----------------------
label: 4
score: 0.0
--------------------- 4586 -----------------------
label: 4
score: 0.3333333333333333
--------------------- 4587 -----------------------
label: 4
score: 0.2

--------------------- 4692 -----------------------
label: 2
score: 0.2857142857142857
--------------------- 4693 -----------------------
label: 3
score: 0.5
--------------------- 4694 -----------------------
label: 4
score: 0.0
--------------------- 4695 -----------------------
label: 1
score: 0.45
--------------------- 4696 -----------------------
label: 3
score: 0.6666666666666666
--------------------- 4697 -----------------------
label: 3
score: 0.3170731707317073
--------------------- 4698 -----------------------
label: 2
score: 0.5
--------------------- 4699 -----------------------
label: 4
score: 0.2857142857142857
--------------------- 4700 -----------------------
label: 3
score: 0.18181818181818185
--------------------- 4701 -----------------------
label: 4
score: 0.26666666666666666
--------------------- 4702 -----------------------
label: 2
score: 0.5
--------------------- 4703 -----------------------
label: 4
score: 0.13333333333333333
--------------------- 4704 ------------

--------------------- 4813 -----------------------
label: 3
score: 0.32
--------------------- 4814 -----------------------
label: 4
score: 0.3333333333333333
--------------------- 4815 -----------------------
label: 1
score: 0.5333333333333333
--------------------- 4816 -----------------------
label: 1
score: 0.4166666666666667
--------------------- 4817 -----------------------
label: 3
score: 0.7
--------------------- 4818 -----------------------
label: 1
score: 0.42857142857142855
--------------------- 4819 -----------------------
label: 1
score: 0.52
--------------------- 4820 -----------------------
label: 2
score: 0.5294117647058824
--------------------- 4821 -----------------------
label: 4
score: 0.4
--------------------- 4822 -----------------------
label: 1
score: 0.5
--------------------- 4823 -----------------------
label: 4
score: 0.29411764705882354
--------------------- 4824 -----------------------
label: 3
score: 0.4666666666666667
--------------------- 4825 ------------

--------------------- 4951 -----------------------
label: 3
score: 0.0
--------------------- 4952 -----------------------
label: 2
score: 0.8181818181818182
--------------------- 4953 -----------------------
label: 4
score: 0.3333333333333333
--------------------- 4954 -----------------------
label: 2
score: 0.4
--------------------- 4955 -----------------------
label: 1
score: 0.8333333333333334
--------------------- 4956 -----------------------
label: 1
score: 0.3
--------------------- 4957 -----------------------
label: 3
score: 0.5333333333333333
--------------------- 4958 -----------------------
label: 4
score: 0.2857142857142857
--------------------- 4959 -----------------------
label: 3
score: 0.6
--------------------- 4960 -----------------------
label: 1
score: 1.0
--------------------- 4961 -----------------------
label: 3
score: 0.3333333333333333
--------------------- 4962 -----------------------
label: 3
score: 0.3
--------------------- 4963 -----------------------
label: 

In [70]:
compare_4 = compare_df[compare_df['overall']==4.0]
compare_3 = compare_df[compare_df['overall']==3.0]
compare_2 = compare_df[compare_df['overall']==2.0]
compare_1 = compare_df[compare_df['overall']==1.0]
len(compare_df)
len(compare_4) + len(compare_3) + len(compare_2) +len(compare_1)

i_list = [1.0,2.0,3.0,4.0]
compare = [compare_1,compare_2,compare_3,compare_4]
for i in i_list:
    index = int(i)-1
    c = compare[index]
    print('Number of pairs that are rated as ',i,':')
    print(len(c))
    print('Pairs that are rated as ',i,', title named entity score is 0: (value, percentage)')
    print(len(c[c['text_ne_score']==0]), ', ', len(c[c['text_ne_score']==0])/len(c)*100,'%')
    print('Pairs that are rated as ',i,', title named entity score is > 0.3:')
    print(len(c[c['text_ne_score']>0.3]), ', ', len(c[c['text_ne_score']>0.3])/len(c)*100,'%')
    print('Pairs that are rated as ',i,', title named entity score is > 0.5:')
    print(len(c[c['text_ne_score']>0.5]), ', ', len(c[c['text_ne_score']>0.5])/len(c)*100,'%')
    print('Pairs that are rated as ',i,', title named entity score is 1.0:')
    print(len(c[c['text_ne_score']==1.0]), ', ', len(c[c['text_ne_score']==1.0])/len(c)*100,'%')

Number of pairs that are rated as  1.0 :
963
Pairs that are rated as  1.0 , title named entity score is 0: (value, percentage)
89 ,  9.241952232606438 %
Pairs that are rated as  1.0 , title named entity score is > 0.3:
749 ,  77.77777777777779 %
Pairs that are rated as  1.0 , title named entity score is > 0.5:
416 ,  43.19833852544133 %
Pairs that are rated as  1.0 , title named entity score is 1.0:
31 ,  3.2191069574247146 %
Number of pairs that are rated as  2.0 :
957
Pairs that are rated as  2.0 , title named entity score is 0: (value, percentage)
108 ,  11.285266457680251 %
Pairs that are rated as  2.0 , title named entity score is > 0.3:
633 ,  66.14420062695925 %
Pairs that are rated as  2.0 , title named entity score is > 0.5:
270 ,  28.213166144200624 %
Pairs that are rated as  2.0 , title named entity score is 1.0:
19 ,  1.9853709508881923 %
Number of pairs that are rated as  3.0 :
1067
Pairs that are rated as  3.0 , title named entity score is 0: (value, percentage)
234 ,  21


### Remarks: <br>

In the case of full text, it is less common to achieve a 1.0 (complete match) even with pairs that are annotated to be similar, as the text has more NEs, and thus, it is less likely for two similar articles to have exactly the same NEs. <br><br>

However, we can observe that texts of articles that have been annotated to be similar seem to have more NEs matching. For example, if we observe the pairs whose scores are >0.3 and >0.5 we can observe that they mirror the annotated similarity score. Moreover, for articles whose similarity score was annotated to be 4.0, there is a very significant correspondence in terms of NEs as 50% of them have assigned overlap similarity precisely equal to 0. <br>

The last piece of information, however, can be read as the confirmation of the fact that titles tend to be most dissimilar because of their purpose, as mentioned previously.

Our conclusion from looking at the NE similarity score from title and text is that since the title score still is consistent with what we would expect, and it's possible that the most important NE are chosen to appear in the title that title named entities can still be a useful indicator.